#LIMPIEZA PROYECTO GOBERNANZA DE DATOS

##Trabajo realizado por:
* Juan Pablo Baquero Velandia
* David Alejandro Dávila Guaranguay

#### NOTA: Este trabajo NO está relacionada a la Biblioteca de la PUJ.
Trabajamos con base en un registro de bibliotecas de Estados Unidos, que da un resumen de la información de entradas y salidas respecto de dinero, recursos, impuestos, etc.

La pregunta que nos va a orientar para hacer el trabajo es acerca de las dimensiones de los datos vistas en clase, para dar métricas y curar los datos para que estas suban.

Para hallar los datos, visitar: [BIBLIOTECAS EE.UU.](https://catalog.data.gov/dataset/public-libraries-b1aaf#:~:text=,File%20CSV)

In [63]:
#Importo las librerías principales para curación de datos

import pandas as pd
import numpy as np
import os

In [64]:
#Importar el archivo y leerlo
df = pd.read_csv("/content/Public_Libraries.csv", sep = ',')
print(df)

      Fiscal Year              Library      County  AENGLC Rank  \
0            1996              Andover     Tolland        109.0   
1            1996              Ansonia   New Haven        158.0   
2            1996      Ashford-Babcock     Windham        131.0   
3            1996                 Avon    Hartford         19.0   
4            1996         Beacon Falls   New Haven        129.0   
...           ...                  ...         ...          ...   
5273         2024           Woodbridge   New Haven         34.0   
5274         2024             Woodbury  Litchfield         43.0   
5275         2024  Woodstock - Bracken     Windham         90.0   
5276         2024      Woodstock - May     Windham         80.0   
5277         2024     Woodstock - West     Windham         90.0   

      Population of Service Area  Total Library Visits  \
0                         2815.0               15000.0   
1                        17825.0               91756.0   
2                    

La página nos proporciona un pequeño diccionario:

* LIBRARY VISITS: Incluye la entrada de todas las personas a la biblioteca, independientemente de su propósito.
* CIRCULATION: Tiene información sobre cualquier material que salga de la biblioteca, incluyendo renovaciones.
* REGISTERED BORROWERS: Son los residentes de un municipio con membresía a la biblioteca.
* REFERENCE QUESTIONS: Habla sobre las interacciones que tiene el personal de la biblioteca con sus usuarios. Recomendaciones, ayuda, etc.
* TOWN TAX APPROPRIATION: Son los fondos dados por el municipio a esa sede de biblioteca.
* AENLGC: Es una métrica basada en los ingresos que tiene un municipio


#EXPLORACIÓN DE LOS DATOS Y SUS DIMENSIONES

## 1. INTEGRIDAD (Y UNICIDAD)
### ¿Los datos están completos?
### -> Borrar duplicados
- - -

In [65]:
# Comencemos por sumar los datos que estén vacíos

df = df.drop_duplicates() # De una vez borramos duplicados

vacios = df.isnull().sum()
print("La cantidad de datos nulos que hay por columna son:\n", vacios)

# De paso puedo saber los nombres de las columnas


La cantidad de datos nulos que hay por columna son:
 Fiscal Year                                              0
Library                                                  0
County                                                   0
AENGLC Rank                                            117
Population of Service Area                             464
Total Library Visits                                   219
Library Visits Per Capita Served                       235
Total Registered Borrowers                             105
Percent of Residents with Library Cards                119
Reference Questions                                    404
Reference Questions Per Capita Served                  433
Total Circulation                                       53
Circulation Per Capita Served                           70
Total Programs (Synchronous + Prerecorded)             168
Total Program Attendance & Views                       191
Total Program Attendance & Views Per Capita Served     213
Use

##### Dandole un vistazo a los registros vacíos, podemos darnos cuenta que todos los datos faltantes sonde tipo entero.
##### Además, los que más faltan son el Uso de internet público, ya que los registros están desde 1996, donde las bibliotecas no normalizaban el uso de internet, además que era caro implementarlo.

In [66]:
#CURACIÓN PROPUESTA

# Aunque sean relevantes, vamos a descartar las columnas de x (per cápita)
# También descartaremos la población objetivo y el acceso a internet.

print(df.shape)

cols_borrar = ['Library Visits Per Capita Served',
               'Reference Questions Per Capita Served',
               'Circulation Per Capita Served',
               'Total Program Attendance & Views Per Capita Served',
               'Use of Public Internet Computers',
               'Collection Per Capita Served',
               'Operating Income Per Capita',
               'Library Materials Expenditures']
# Esto me dió un error de media hora 😭

# Borraremos aquellos registros con más del 20% de la información faltante
# Es un filtro algo fuerte, pero da cuenta de

df = df.dropna(thresh = int(df.shape[1] * 0.8))

df = df.drop(columns = cols_borrar, errors='ignore')

print("Las columnas que quedan son:\n", df.columns.to_list())

print(df.shape)

(5278, 27)
Las columnas que quedan son:
 ['Fiscal Year', 'Library', 'County', 'AENGLC Rank', 'Population of Service Area', 'Total Library Visits', 'Total Registered Borrowers', 'Percent of Residents with Library Cards', 'Reference Questions', 'Total Circulation', 'Total Programs (Synchronous + Prerecorded)', 'Total Program Attendance & Views', 'Total Collection', 'Total Operating Income', 'Town Tax Appropriation for Library', 'Tax Appropriation Per Capita Served', 'Wages & Salaries Expenditures', 'Operating Expenditures', 'Operating Expenditures Per Capita']
(5133, 19)


- - -
## 2. PUNTUALIDAD Y UNIFORMIDAD

#### En este caso, los datos están accesibles de forma sencilla y rápida, porque son relativamente pocos registros.

#### Por parte de la uniformidad, no hay ningún conflicto, ya que el dataframe es todo de tipo entero y se puede verificar facilmente que no tiene errores de ortografía en las casillas de strings o cosas por el estilo.
- - -

## 3. VALIDEZ

### Vamos a revisar el formato de los datos y su rango
- - -

In [67]:
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
num_cols

Index(['Fiscal Year', 'AENGLC Rank', 'Population of Service Area',
       'Total Library Visits', 'Total Registered Borrowers',
       'Percent of Residents with Library Cards', 'Reference Questions',
       'Total Circulation', 'Total Programs (Synchronous + Prerecorded)',
       'Total Program Attendance & Views', 'Total Collection',
       'Total Operating Income', 'Town Tax Appropriation for Library',
       'Tax Appropriation Per Capita Served', 'Wages & Salaries Expenditures',
       'Operating Expenditures', 'Operating Expenditures Per Capita'],
      dtype='object')

####Como podemos ver, todas menos las primeras son de tipo numérico

##  4. EXACTITUD
### La idea ahora es rellenar los datos faltantes con la mediana, que es un proceso muy sencillo pero eficaz

In [68]:
# Vamos a reemplazar los faltantes por la mediana
# Uso el mismo método que la vez pasada

for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    mediana = df[col].median()
    df[col] = df[col].apply(lambda x: mediana if x < limite_inferior or x > limite_superior else x)

print("Outliers reemplazados correctamente con la mediana.")

Outliers reemplazados correctamente con la mediana.


# CREACIÓN DE MÉTRICAS SOBRE EL DATASET



> Funciones/columnas creadas con ayuda de ChatGPT.

> Autores del resto del código: Juan Pablo y David Alejandro

> Fecha: 20-10-2025

> Propósito: Métricas para análisis exploratorio de bibliotecas.





In [69]:
# =============================================
# Cálculos de métricas y nuevas columnas
# Generado con ayuda de ChatGPT
# =============================================

df['Visits_per_Borrower'] = df['Total Library Visits'] / df['Total Registered Borrowers']
df['References_per_Borrower'] = df['Reference Questions'] / df['Total Registered Borrowers']
df['Population_Group'] = pd.cut(
    df['Population of Service Area'],
    bins=[0, 10000, 50000, 200000, 1000000],
    labels=['Pequeña', 'Mediana', 'Grande', 'Muy Grande']
)
df['Log_Population'] = np.log1p(df['Population of Service Area'])
df['Program_Attendance_per_Capita'] = df['Total Program Attendance & Views'] / df['Population of Service Area']


#RESUMEN

In [70]:
print("Filas y columnas:", df.shape)
print(df.head()) # Una muestra
print(df.dtypes) # Revisar los tipos de datos
print(df.isna().sum()) # Revisar nulos

# Revisamos consistencia de las nuevas columnas agregadas
print(df['Visits_per_Borrower'].describe())
print(df['Population_Group'].value_counts())



Filas y columnas: (5133, 24)
   Fiscal Year          Library     County  AENGLC Rank  \
0         1996          Andover    Tolland        109.0   
1         1996          Ansonia  New Haven        158.0   
2         1996  Ashford-Babcock    Windham        131.0   
3         1996             Avon   Hartford         19.0   
4         1996     Beacon Falls  New Haven        129.0   

   Population of Service Area  Total Library Visits  \
0                      2815.0               15000.0   
1                     17825.0               91756.0   
2                      3969.0                8665.0   
3                     14143.0              162893.0   
4                      5351.0                7000.0   

   Total Registered Borrowers  Percent of Residents with Library Cards  \
0                      1313.0                                     0.47   
1                      6952.0                                     0.39   
2                      1360.0                                  

In [71]:
#GUARDADO DEL ARCHIVO LIMPIO
df.columns = df.columns.str.replace(' ', '_')  # Reemplaza espacios por '_', para que se guarde bien

df.to_csv("DataSetLimpio.csv", index = False)

print("DataSetLimpio.csv creado exitosamente!!")

DataSetLimpio.csv creado exitosamente!!
